# **Análise de Sentimentos com RNN**

Instituto Mauá de Tecnologia

Giovanni Schiffini - 2020


Montando Google Drive e importando bibliotecas:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB


Importando CSV das análises PT-BR do IMDB:

In [ ]:
imdb = pd.read_csv('/content/drive/My Drive/Colab Notebooks/imdb-reviews-pt-br.csv',encoding='utf-8')

In [ ]:
imdb_pt = imdb[['text_pt','sentiment']]
print("Cabeçalho dos dados:")
imdb_pt.head()

Cabeçalho dos dados:


,text_pt,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,Este é um exemplo do motivo pelo qual a maiori...,neg
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,Filmes de fotos de latão não é uma palavra apr...,neg


In [ ]:
print("Término dos dados:")
imdb_pt.tail()

Término dos dados:


,text_pt,sentiment
49454,"Como a média de votos era muito baixa, e o fat...",pos
49455,O enredo teve algumas reviravoltas infelizes e...,pos
49456,Estou espantado com a forma como este filme e ...,pos
49457,A Christmas Together realmente veio antes do m...,pos
49458,O drama romântico da classe trabalhadora do di...,pos


# **Tratamento dos Dados**

Transformando análises negativas em ZERO e análises positivas em UM:

In [ ]:
imdb_pt.sentiment.unique()

array(['neg', 'pos'], dtype=object)

In [ ]:
imdb_pt.sentiment.replace( {'neg': 0,'pos': 1}, inplace=True )

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [ ]:
print("Conjunto de dados com alteração do sentiment para números 0 e 1:")
imdb_pt

Conjunto de dados com alteração do sentiment para números 0 e 1:


,text_pt,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",0
1,Este é um exemplo do motivo pelo qual a maiori...,0
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",0
3,Nem mesmo os Beatles puderam escrever músicas ...,0
4,Filmes de fotos de latão não é uma palavra apr...,0
...,...,...
49454,"Como a média de votos era muito baixa, e o fat...",1
49455,O enredo teve algumas reviravoltas infelizes e...,1
49456,Estou espantado com a forma como este filme e ...,1
49457,A Christmas Together realmente veio antes do m...,1


Informações de texto para o eixo X e de sentimento para o eixo Y:

In [ ]:
imdb_reviews = imdb_pt.text_pt.values
imdb_sentiment =imdb_pt.sentiment.values   

Tokenização dos textos: separação das frases em palavras unitárias, para facilitar a análise de NLP.

In [ ]:
imdb_reviews_token = []

for sent in imdb_reviews:
  sentenca= nltk.word_tokenize(sent)
  imdb_reviews_token += [sentenca]


In [ ]:
imdb_reviews_token[0]

['Mais',
 'uma',
 'vez',
 ',',
 'o',
 'Sr.',
 'Costner',
 'arrumou',
 'um',
 'filme',
 'por',
 'muito',
 'mais',
 'tempo',
 'do',
 'que',
 'o',
 'necessário',
 '.',
 'Além',
 'das',
 'terríveis',
 'seqüências',
 'de',
 'resgate',
 'no',
 'mar',
 ',',
 'das',
 'quais',
 'há',
 'muito',
 'poucas',
 ',',
 'eu',
 'simplesmente',
 'não',
 'me',
 'importei',
 'com',
 'nenhum',
 'dos',
 'personagens',
 '.',
 'A',
 'maioria',
 'de',
 'nós',
 'tem',
 'fantasmas',
 'no',
 'armário',
 ',',
 'e',
 'o',
 'personagem',
 'Costers',
 'é',
 'realizado',
 'logo',
 'no',
 'início',
 ',',
 'e',
 'depois',
 'esquecido',
 'até',
 'muito',
 'mais',
 'tarde',
 ',',
 'quando',
 'eu',
 'não',
 'me',
 'importava',
 '.',
 'O',
 'personagem',
 'com',
 'o',
 'qual',
 'deveríamos',
 'nos',
 'importar',
 'é',
 'muito',
 'arrogante',
 'e',
 'superconfiante',
 ',',
 'Ashton',
 'Kutcher',
 '.',
 'O',
 'problema',
 'é',
 'que',
 'ele',
 'sai',
 'como',
 'um',
 'garoto',
 'que',
 'pensa',
 'que',
 'é',
 'melhor',
 'do',
 

Preparação dos conjuntos de treino e teste:

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(imdb_reviews,imdb_sentiment,test_size=0.2,random_state=21)

In [ ]:
print("Conjunto de teste - Eixo Y - Sentimento (0/1):")
y_test

Conjunto de teste - Eixo Y - Sentimento (0/1):


array([1, 0, 0, ..., 1, 0, 1])

In [ ]:
print("Conjunto de teste - Eixo X - Textos:")
x_test

Conjunto de teste - Eixo X - Textos:


array(['Soylent Green é um clássico. Eu estive esperando por alguém para refazer isso. Eles parecem estar refazendo clássicos da ficção científica nos dias de hoje, como Guerra dos Mundos, e eu espero que algum diretor / produtor re-faça o Soylent Green. Com a animação e tecnologia de computadores de hoje, ela teria o potencial de ser uma ótima foto. Filmes anti-utópicos podem não ser tão absurdos. A raça humana se reproduz como baratas sem influência externa para reduzi-la. Nós, como seres humanos, temos a opção de colocar o kibosh na procriação de espécies menores, se eles saem do controle, mas não há nada para controlar a criação humana, exceto para nós mesmos. Apesar de todas as doenças, guerras, abortos, controle de natalidade, etc. a raça humana ainda se multiplica como bactérias em uma placa de Petri. A clássica economia malthusiana afirma que qualquer espécie, incluindo os humanos, se multiplicará além de seus meios de subsistência. 6 bilhões e crescendo .... isso é obsceno.',


# **Bag of Words**

"Bag of Words" é um modelo utilizado como ferramenta para geração de features em uma rede. As palavras que aparecem com mais frequência no saco-de-palavras tem maior influência sobre os features.

Stopwords: lista de palavras como "o", "a", "de", "e", etc, que serão removidas do conjunto de dados para não prejudicar os features:


In [ ]:
nltk.download('stopwords')
stopwords_pt = nltk.corpus.stopwords.words('portuguese')
stopwords_pt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['de',
 'a',
 'o',
 'que',
 'e',
 'é',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'não',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'à',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'nos',
 'já',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'ela',
 'entre',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'você',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'às',
 'minha',
 'numa',
 'pelos',
 'elas',
 'qual',
 'nós',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'dele',
 'tu',
 'te',
 'vocês',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo',
 'estou',
 'está',
 'estamos',
 'estão',
 'estive',
 'esteve',
 'estivemos',
 'estiveram',
 'estava',
 'estávamos',
 'estavam',
 'estivera'

Vetorização das palavras sem os stopwords:

In [ ]:
vectorize = CountVectorizer(analyzer = 'word', stop_words= stopwords_pt)
vectorize.fit(imdb_reviews)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em',
                            'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se',
                            'na', 'por', 'mais', 'as', 'dos', 'como', 'mas',
                            'ao', 'ele', 'das', 'à', 'seu', 'sua', ...],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

Separação dos dados em treino e teste:

In [ ]:
bow_x_train = vectorize.transform(x_train)
bow_x_test  = vectorize.transform(x_test)

In [ ]:
print("Conjunto de palavras para treino:")
bow_x_train

Conjunto de palavras para treino:


<39567x129446 sparse matrix of type '<class 'numpy.int64'>'
	with 4060238 stored elements in Compressed Sparse Row format>

In [ ]:
print("Conjunto de palavras para teste:")
bow_x_test

Conjunto de palavras para teste:


<9892x129446 sparse matrix of type '<class 'numpy.int64'>'
	with 1011982 stored elements in Compressed Sparse Row format>

Nota-se uma proporção adequada de 5.3 milhão de palavras para treino e 1.3 milhão de palavras para teste.

##Classificador

Construção do classificador de sentimentos:

In [ ]:
model = MultinomialNB()
model.fit(bow_x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# **Predição de sentimentos em comentários**

In [ ]:
class color:
   GREEN = '\033[92m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   END = '\033[0m'

print(color.BOLD + "Insira sua opinião sobre o filme: \n" + color.END)
comentario = [input()]
vetorizado = vectorize.transform(comentario).toarray()
predicao = model.predict(vetorizado)
if ((predicao) == 1):
  print(color.GREEN + "\nEste é um comentário positivo" + color.END)
else:
  print(color.RED + "\nEste é um comentário negativo" + color.END)

Insira sua opinião sobre o filme: 

Apesar de algumas falhas, o filme me agradou.

Este é um comentário positivo


In [ ]:
class color:
   GREEN = '\033[92m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   END = '\033[0m'

print(color.BOLD + "Insira sua opinião sobre o filme: \n" + color.END)
comentario = [input()]
vetorizado = vectorize.transform(comentario).toarray()
predicao = model.predict(vetorizado)
if ((predicao) == 1):
  print(color.GREEN + "\nEste é um comentário positivo" + color.END)
else:
  print(color.RED + "\nEste é um comentário negativo" + color.END)

Insira sua opinião sobre o filme: 

Apesar de ter bons atores, o filme é ruim.

Este é um comentário negativo
